In [1]:
import json
import numpy as np

with open('cloverleaf.pbf.json') as f:
    geojson = json.load(f)

In [2]:
polylines = [f['geometry']['coordinates'] for f in geojson['features'] if f['properties'].get('type') == 'lane_border']
print(f'#polylines: {len(polylines)}')

# convert to ENU
from polyline_ruler.tf import lla2enu
anchor = polylines[0][0]
polylines = [lla2enu(llas, anchor_lla=anchor).astype(np.float32) for llas in polylines]

#polylines: 808


In [3]:
from fast_crossing import FastCrossing

fc = FastCrossing()
for line in polylines:
    fc.add_polyline(line)
fc.finish()

In [4]:
intersections = fc.intersections()

In [5]:
print(len(intersections))
print(intersections[0]) # intersection point, (t, s), (polyline_idx, segment_idx), (polyline_idx, segment_idx), 

print('poles')
poles = []
for inter in intersections:
    xy, (t, s), (poly1, seg1), (poly2, seg2) = inter
    if (t, s) in ((0, 0), (0, 1), (1, 0), (1, 1)):
        continue
    xyz1 = fc.coordinates(poly1, seg1, t)
    xyz2 = fc.coordinates(poly2, seg2, s)
    if np.fabs(xyz1[2] - xyz2[2]) < 1e-3:
        continue
    # print(inter)
    # print(xy, (t, s))
    # print(xyz1)
    # print(xyz2)
    poles.append([xyz1, xyz2])
print(f'#poles: {len(poles)}')
    

1042
(array([0., 0.]), array([0., 1.]), array([0, 0], dtype=int32), array([312,   1], dtype=int32))
poles
#poles: 239


In [6]:
import k3d
import numpy as np

plot = k3d.plot()
for polyline in polylines:
    plot += k3d.line(polyline)
for line in poles:
    plot += k3d.line(line, color=0xff0000)
    
plot.display()

Output()